In [1]:
from p2pfl.node import Node
import time

In [2]:
n=5
nodes = []
for i in range(n):
    node = Node(host="localhost",model=i)
    node.start()
    nodes.append(node)
    
for node in nodes:
    print(node.model)


INFO:root:Nodo a la escucha en localhost 65404
INFO:root:Nodo a la escucha en localhost 65405
INFO:root:Nodo a la escucha en localhost 65406
INFO:root:Nodo a la escucha en localhost 65407
INFO:root:Nodo a la escucha en localhost 65408
0
1
2
3
4


In [3]:
for i in range(len(nodes)-1):
    nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
    time.sleep(0.1)


INFO:root:Conexión aceptada con ('localhost', 65405)
INFO:root:Conexión aceptada con ('localhost', 65406)
INFO:root:Conexión aceptada con ('localhost', 65404)
INFO:root:Conexión aceptada con ('localhost', 65407)
INFO:root:Conexión aceptada con ('localhost', 65405)
INFO:root:Conexión aceptada con ('localhost', 65404)
INFO:root:Conexión aceptada con ('localhost', 65408)
INFO:root:Conexión aceptada con ('localhost', 65405)
INFO:root:Conexión aceptada con ('localhost', 65406)
INFO:root:Conexión aceptada con ('localhost', 65404)


In [4]:
# Check if they are connected
for node in nodes:
    print(len(node.neightboors) == n-1)
    

True
True
True
True
True


In [5]:
nodes[0].set_start_learning()

INFO:root:Broadcasting model to all clients: 1
INFO:root:Broadcasting model to all clients: 2
INFO:root:Broadcasting model to all clients: 0
INFO:root:Broadcasting model to all clients: 3
DEBUG:root:Processing params: PARAMS 1 \PARAMS
DEBUG:root:Processing params: PARAMS 1 \PARAMS
INFO:root:Broadcasting model to all clients: 4
DEBUG:root:Processing params: PARAMS 1 \PARAMS
INFO:root:Model added (1/4)
DEBUG:root:Processing params: PARAMS 2 \PARAMS
INFO:root:Model added (1/4)
DEBUG:root:Processing params: PARAMS 2 \PARAMS
DEBUG:root:Processing params: PARAMS 2 \PARAMS
DEBUG:root:Processing params: PARAMS 0 \PARAMS
DEBUG:root:Processing params: PARAMS 0 \PARAMS
DEBUG:root:Processing params: PARAMS 3 \PARAMS
DEBUG:root:Processing params: PARAMS 3 \PARAMS
DEBUG:root:Processing params: PARAMS 0 \PARAMS
INFO:root:Model added (2/4)
DEBUG:root:Processing params: PARAMS 3 \PARAMS
INFO:root:Model added (1/4)
INFO:root:Model added (1/4)
DEBUG:root:Processing params: PARAMS 4 \PARAMS
DEBUG:root:Pro

In [7]:
for node in nodes:
    print(node.model)

2.0
2.0
2.0
2.0
2.0


In [7]:
for node in nodes:
    print(node.round)

1
1
1
1
1


In [10]:
for node in nodes:
    print(node.model)

0
1
2
3
4


In [11]:
nodes[0].set_stop_learning()

INFO:root:Stopping learning
INFO:root:Stopping learning
INFO:root:Stopping learning
INFO:root:Stopping learning
INFO:root:Stopping learning


In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import pytorch_lightning as pl
from torchmetrics import Accuracy

class MLP(pl.LightningModule):


    def __init__(self):
        super().__init__()
        self.lr_rate = 0.02
        self.accuracy = Accuracy()

        # 10 clases de salida
        self.l1 = torch.nn.Linear(28 * 28, 128)
        self.l2 = torch.nn.Linear(128, 256)
        self.l3 = torch.nn.Linear(256, 10)

    def forward(self, x):
        batch_size, channels, width, height = x.size()
        
        # (b, 1, 28, 28) -> (b, 1*28*28)
        x = x.view(batch_size, -1)

        x = self.l1(x)
        x = torch.relu(x)
        x = self.l2(x)
        x = torch.relu(x)
        x = self.l3(x)


        x = torch.log_softmax(x, dim=1)
        return x

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr_rate)


    def training_step(self, batch, batch_id):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        
        self.log("train_loss", loss, prog_bar=True)

        return loss 


    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)

        loss = F.cross_entropy(self(x), y)

        out = torch.argmax(logits, dim=1)
        acc = self.accuracy(out, y) 

        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        
        self.log
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)

        loss = F.cross_entropy(self(x), y)

        out = torch.argmax(logits, dim=1)
        acc = self.accuracy(out, y) 

        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", acc, prog_bar=True)
        
        self.log
        return loss
        


In [2]:
model = MLP()

In [6]:
[val.cpu().numpy() for _, val in model.state_dict().items()]


[array([[-0.02893332,  0.02919314,  0.01222849, ...,  0.01862139,
          0.02185085,  0.02695582],
        [-0.03463912,  0.00209042, -0.03030546, ...,  0.00085545,
          0.01175655,  0.0272004 ],
        [ 0.03290805, -0.00562747,  0.0301982 , ...,  0.03348413,
         -0.00771396, -0.0193994 ],
        ...,
        [ 0.01508801, -0.00857255,  0.03219221, ...,  0.0045099 ,
          0.00885244, -0.0155958 ],
        [-0.03231334, -0.01538485, -0.03044182, ...,  0.01606286,
          0.00039962, -0.02985382],
        [-0.01017734, -0.03229888,  0.01801966, ...,  0.00917631,
         -0.01646002, -0.01879791]], dtype=float32),
 array([-5.87676652e-03, -7.94191100e-03, -1.20719597e-02,  2.04666555e-02,
         2.60983855e-02, -2.34108493e-02, -3.17336768e-02,  3.10098194e-02,
        -2.68743522e-02,  2.63165981e-02, -1.04977619e-02, -2.83281524e-02,
        -2.26684734e-02, -2.33334303e-03, -1.77407824e-02,  7.55000487e-03,
        -1.85791440e-02, -1.95869934e-02, -1.26519632e